<h2>Project introduction</h2>

In [21]:
# This Notebook runs the power flow of KU distribution system and shows the 
# network on an interactive open street map

# Nomenclature
# -----------------
# HVB -> High voltage Bus
# LVB1 -> Low voltage Bus number 1
# Line1_2 -> Line connected between LVB1(low voltage bus number 1) and LVB2(low voltage bus number 2) 
# Load3 -> Load connected to LVB3(low voltage bus number 3)

# Parameters
# -------------------
# line resistance = 0.082 ohm/km
# line reactance = 0.07 ohm/km
# power factor = 0.95

# External grid: A generator has been modelled as an external grid, its control is set to Slack

<h2>Import Dependencies</h2>

In [22]:
import pypsa
import pandas as pd
import matplotlib.pyplot as plt
import numpy  as np
import cartopy.crs as ccrs
import math
import matplotlib.ticker as mticker
import folium
import time 
import paho.mqtt.client as mqtt
import json
import ku_grid
import threading
from folium.plugins import AntPath

<h2>Create Network</h2>

In [23]:
# ku_grid.py contains the distribution network model of KU
# caling ku_grid.create_network() will create and return the network
network = ku_grid.create_network()

In [24]:
network.consistency_check()

<h2>Load Flow & Network Plot</h2>

In [25]:
# This section first fetches data every minute from WiFi energy meters installed in each block by using 
# MQTT protocol. Total three phase power is retrieved from each meter and p_set and q_set of the 
# corresponding loads are updated in the network. It then performs the NR load flow of the network and plots the result.
# Python's Folium library created the open street map and saves it in HTML format

<h3>Set up for Geographic Map</h3>

In [26]:
# path to save map
MAP_PATH = r"G:\My Drive\D-VA\Main Project\ku_grid.html"

# create a map object
map = folium.Map(location=(27.619013147338894, 85.5387356168638), 
                    zoom_start=17, max_zoom=30)

# add a layer to plot the distribution grid
grid_layer = folium.FeatureGroup(name='Grid Layer').add_to(map)

# add a layer for animation
animation_layer = folium.FeatureGroup(name='Animation').add_to(map)
folium.LayerControl().add_to(map)

# get coordinates of all the buses in the network
bus_coords = []
for index, row in network.buses.iterrows():
#     first latitude then longitude as folium expects location in this order
    bus_coords.append([row['y'], row['x']])

# Define a legend for the buses
bus_legend_html = """
        <div style="position: fixed; 
        top: 300px; right: 50px; width: 150px; height: 180px; 
        border:0px solid grey; z-index:9999; font-size:14px;
        background-color: white;
        ">&nbsp; <span style="font-weight: bold; font-size: 20px">Bus Legends </span></b><br>
        &nbsp; <font color="red" style="font-size: 30px;">●</font><span style="font-weight:bold;"> |V| < 0.95</span>   <br>
        &nbsp; <font color="green" style="font-size: 30px;">●</font><span style="font-weight:bold;"> 0.95 ≤ |V| ≤ 1.05</span><br>
        &nbsp; <font color="yellow" style="font-size: 30px;">●</font><span style="font-weight:bold;"> 1.05 < |V|</span><br>
        </div>
        """

# Define a legend for the lines
line_legend_html = """
        <div style="position: fixed; 
        bottom: 20px; right: 20px; width: 200px; height: 180px; 
        border:0px solid grey; z-index:9999; font-size:14px;
        background-color: white;
        ">&nbsp; <span style="font-weight: bold; font-size: 20px">Line Legends </span></b><br>
        &nbsp; <font color="green" style="font-size: 30px;">—</font><span style="font-weight:bold;"> Loading ≤ 50%</span><br>
        &nbsp; <font color="orange" style="font-size: 30px;">—</font><span style="font-weight:bold;"> 50% ≤ Loading < 100%</span><br>
        &nbsp; <font color="red" style="font-size: 30px;">—</font><span style="font-weight:bold;"> Loading > 100%</span><br>
        </div>
        """
# Add bus legend to the map
map.get_root().html.add_child(folium.Element(bus_legend_html))

# Add line legend to the map
map.get_root().html.add_child(folium.Element(line_legend_html))


<h3>Set up for MQTT communication with the Iammeter broker</h3>

In [27]:
# MQTT broker details
broker_address = "mqtt.iammeter.com"  # MQTT broker address - iammeter broker in this case
broker_port = 1883  # Default MQTT port
username = "karuna"
password = "232794"

TOTAL_BLOCKS = 4

PHYSICS = 0
BIOTECH = 1
MANAGEMENT = 2
CIVIL = 3

# Topics to subscribe to, for each meter
Topic = {PHYSICS: "device/CD0FF6AB/realtime",
          BIOTECH: "device/57DB095D/realtime",
          MANAGEMENT: "device/8FA834AC/realtime",
          CIVIL: "device/DAD94549/realtime"}


# set the power factor of 0.95
PF = 0.95
tan_phi = math.sqrt(1-PF**2)/PF

# initialize the variables to store the total power for each block
physics_meter_total_power = 1000
biotech_meter_total_power = 1000
management_meter_total_power = 1000
civil_meter_total_power = 1000

# Callback function to handle incoming messages
def on_message(client, userdata, message):
    # decode the message into a python string and then convert to a dictionary  
    Payload_str = message.payload.decode("utf-8")
    payload_dict = json.loads(Payload_str)

    # get the active powers of all three phases
    pa = int(payload_dict['Datas'][0][2])
    pb = int(payload_dict['Datas'][1][2])
    pc = int(payload_dict['Datas'][2][2])
    total_power = pa+pb+pc

    # store the total power to the variables of corresponding blocks
    global physics_meter_total_power
    global biotech_meter_total_power
    global civil_meter_total_power
    global management_meter_total_power

    global map
    global grid_layer

    if message.topic == Topic[PHYSICS]:
        physics_meter_total_power = total_power
        network.loads.loc['Load16', 'p_set'] = physics_meter_total_power/1e6
        network.loads.loc['Load16', 'q_set'] = (physics_meter_total_power/1e6)*tan_phi
    elif message.topic == Topic[BIOTECH]:
        biotech_meter_total_power = total_power
        network.loads.loc['Load19', 'p_set'] = biotech_meter_total_power/1e6
        network.loads.loc['Load19', 'q_set'] = (biotech_meter_total_power/1e6)*tan_phi
    elif message.topic == Topic[MANAGEMENT]:
        management_meter_total_power = total_power
        network.loads.loc['Load5', 'p_set'] = management_meter_total_power/1e6
        network.loads.loc['Load5', 'q_set'] = (management_meter_total_power/1e6)*tan_phi
    elif message.topic == Topic[CIVIL]:
        civil_meter_total_power = total_power
        network.loads.loc['Load6', 'p_set'] = civil_meter_total_power/1e6
        network.loads.loc['Load6', 'q_set'] = (civil_meter_total_power/1e6)*tan_phi
 

<h3>Load Flow and Network Plotting</h3>

In [ ]:
# perform NR load flow and plot the network
def load_flow():
    global physics_meter_total_power
    global biotech_meter_total_power
    global civil_meter_total_power
    global management_meter_total_power
    while True:
        # perform newton Raphson Load Flow
        network.pf()

        ####################################################################
        ######################### Network Plotting #########################
        ####################################################################
    
        # add circles to the locations of buses in the map
        for i in range(len(bus_coords)):
            # get the bus name
            bus_name = network.buses.index.to_list()[i]
            # get per unit voltage magnitude the bus
            v_mag_pu = network.buses_t.v_mag_pu.iloc[0, i]
            # get voltage angle of the bus (in radian by default) and convert it to degree
            v_ang_rad = network.buses_t.v_ang.iloc[0, i]
            v_ang_deg = (180/math.pi)*v_ang_rad 
            # set bus color based on voltage magnitude
            bus_color=''
            if v_mag_pu<0.95:
                bus_color='red'
            elif 0.95<=v_mag_pu<=1.05:
                bus_color='green'
            else:
                bus_color='yellow'
            # show bus voltage magnitude and voltage angle on the popup 
            popup_text = f'<span style="font-weight:bold; padding-left:20px;">{bus_name}</span><br>|V| = {v_mag_pu: .3f} p.u.<br>δ = {v_ang_deg: .3f} deg'
            folium.Circle(location=bus_coords[i], radius=3.5, 
                        stroke=False,
                        fill=True, fill_color= bus_color, fill_opacity=1.0,
                        popup=folium.Popup(popup_text, max_width=100)).add_to(grid_layer)
    
        # add lines
        for index, row in network.lines.iterrows():
            # get the name of the line
            line_name = index
            # get active and reactive powers of the line
            line_p = network.lines_t.p0.loc['now', index ]
            line_q = network.lines_t.q0.loc['now', index ]    
            # get the starting and ending buses of each line
            bus0 = row['bus0']
            bus1 = row['bus1']
            # set line colors based on the line loading
            # assume nominal line apparent capacity of 0.4 MVA
            s_nom_assumed = 0.069   #assumed nominal capacity of the line (230*300/1000000 MVA)
            # calculate the line percentage loading
            percentage_loading = (abs(network.lines_t.p0.loc['now', index ])/s_nom_assumed)*100
            line_color = ''
            if percentage_loading <= 50:
                # green color if line loading is less than 50%
                line_color = 'green' 
            elif (50 < percentage_loading <= 100):
                # violet if line loading is between 50 to 100%
                line_color = 'orange' 
            else:
                # red if line loading is greater than 100%
                line_color = 'red'

            # set line weight relative to percentage loading
            line_weight = 2.0 + percentage_loading*4/100
            # tooltip text for the line
            tooltip_text = f'<span style="font-weight: bold; padding-left: 0px">{line_name}</span><br>P = {line_p: .3f} MW<br>Q = {line_q:.3f} MVAr<br>loading = {percentage_loading: .3f}%'
            # finally, add the line
            # latitude first then longitude
            folium.PolyLine(locations=[(network.buses.loc[bus0].y, network.buses.loc[bus0].x), 
                                    (network.buses.loc[bus1].y, network.buses.loc[bus1].x)],
                            color = line_color, weight  = line_weight,
                        tooltip= tooltip_text).add_to(grid_layer)
            
            x1, y1 = network.buses.loc[bus0].x, network.buses.loc[bus0].y  #first point of the line
            x2, y2 = network.buses.loc[bus1].x, network.buses.loc[bus1].y  #second point
            x3, y3 = (x1+x2)/2, (y1+y2)/2     # mid point
            m = (y2-y1)/(x2-x1)     #slope
            l = math.sqrt(pow(x2-x1, 2) + pow(y2-y1, 2))    #line length
            al = l/8    #arrow length
            # print(f'{line_name}: slope = {m}  & length = {l}')
            theta = math.atan(m)
            theta = abs(theta)
            phi = math.pi/8     # angle between the main line and the arrow lines 
            p = al*math.sin(theta)
            b = al*math.cos(theta)
            p1= al*math.tan(phi)
            b1= p1*math.cos(theta)
            k1= b1*math.tan(theta)
            p2 = p1
            b2 = b1
            k2 = k1
            if (x1<x2) and (y1<y2):
                # coordinates for arrowheads to the lines having positive slope, arrowhead pointing upwards
                xprime=x3-b
                yprime=y3-p
                x4=xprime-k1
                y4=yprime+b1
                x5=xprime+k2
                y5=yprime-b2

            elif (x1<x2) and (y1>y2):
                 # coordinates for arrowheads to the lines having negative slope, arrowhead pointing downwards
                xprime=x3-b
                yprime=y3+p
                x4=xprime+k1
                y4=yprime+b1
                x5=xprime-k2
                y5=yprime-b2

            elif (x1>x2) and (y1<y2):
                # coordinates for arrowheads to the lines having negative slope, arrowhead pointing upwards
                xprime=x3+b
                yprime=y3-p
                x4=xprime-k1
                y4=yprime-b1
                x5=xprime+k2
                y5=yprime+b2

            elif (x1>x2) and (y1>y2):
                # coordinates for arrowheads to the lines having positive slope, arrowhead pointing downwards
                xprime=x3+b
                yprime=y3+p
                x4=xprime+k1
                y4=yprime-b1
                x5=xprime-k2
                y5=yprime+b2

            # use triangles as arrowheads
            folium.Polygon(locations=[(y4, x4), (y3, x3), (y5, x5)],
                     color= line_color, weight=2.0,
              fill=True, fill_color = line_color, fill_opacity=0.8).add_to(grid_layer)

            # Use AntPath for animation
            # coordinates - first latitude(y) then longitude(x)
            AntPath([(y1, x1), (y2, x2)], 
                    delay = 400, dash_array=(3,10), 
                    color=line_color, pulse_color='#FFFFFF',
                    weight=3, opacity=1.0).add_to(animation_layer)
    
        # add a line between HVB and LVB1 as PyPSA doesn't create a line between the buses if there is a transformer in between
        folium.PolyLine(locations=[(network.buses.loc['HVB'].y, network.buses.loc['HVB'].x), 
                                    (network.buses.loc['LVB1'].y, network.buses.loc['LVB1'].x)],
                                color = 'black').add_to(grid_layer)

        # save the geomap of the network in an html file
        map.save(MAP_PATH)
        
    #     Autorefresh section -- modify the html file so that it autorefreshes every minute
        with open(MAP_PATH, 'r', encoding='utf-8') as f:
            f_contents = f.read()
        
        refreshed_content = f_contents.replace('</head>', '<meta http-equiv="refresh" content="60"></head>')
    
        with open(MAP_PATH, 'w', encoding='utf-8') as f:
            f.write(refreshed_content)
        
        time.sleep(60)

#create a thread to handle the data operations
#so that data fetching and manipulation runs independently
thread = threading.Thread(target=load_flow)
thread.start()

# Create MQTT client instance
client = mqtt.Client()

# Set username and password for authentication
client.username_pw_set(username, password)

# Assign callback function to handle incoming messages
client.on_message = on_message

# Connect to MQTT broker
client.connect(broker_address, broker_port)

# Subscribe to the topics
client.subscribe(Topic[PHYSICS])
client.subscribe(Topic[BIOTECH])
client.subscribe(Topic[MANAGEMENT])
client.subscribe(Topic[CIVIL])

# Loop to maintain MQTT connection and process incoming messages
client.loop_forever()



INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 2 iterations with error of 0.000000 in 0.015877 seconds
INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 2 iterations with error of 0.000000 in 0.014243 seconds
INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 2 iterations with error of 0.000000 in 0.016024 seconds
INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 2 iterations with error of 0.000000 in 0.012955 seconds
INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network 

<h2>Data Extration through API - No Longer Used</h2>

In [ ]:
# This section first fetches data every 5 minutes from WiFi energy meters installed in each block by using 
# the API provided by iammeter. Total three phase power is retrieved from each meter and p_set of the 
# corresponding loads is updated in the network.It then performs the NR load flow of the network and plots the result.
# All of these processes are done inside an infinite while loop for real time visualization.
# It uses folium library for plotting the network

In [ ]:
# import requests
# import json
# import time
# from IPython.display import display, clear_output

# TOTAL_BLOCKS = 4

# PHYSICS = 0
# BIOTECH = 1
# MANAGEMENT = 2
# CIVIL = 3

# # set wait time(in seconds) before making another API call
# WAIT_TIME = 5*60

# # SN of energy meters installed in each block
# # create a list of random strings for initialization
# SN = ["initial_SN_string" for _ in range(TOTAL_BLOCKS)]

# # assign SN of individual meters
# SN[PHYSICS] = 'CD0FF6AB'
# SN[BIOTECH] = '57DB095D'
# SN[MANAGEMENT] = '8FA834AC'
# SN[CIVIL] = 'DAD94549'

# # TOKEN to be used for making api call
# TOKEN = 'a971224e6dc04edfa62bc0e83978d854'

# # create URLs to make API calls to fetch data from the Iammeter Cloud
# URL = []
# for i in range(TOTAL_BLOCKS):
#     URL.append("http://www.iammeter.com/api/v1/site/meterdata2/" + SN[i] + "?token=" + TOKEN)


# class Meter:
#     def __init__(self, url: str):
#         self.url = url
#         # print(self.url)

#     def get_meterdata(self) -> object:
#         response = requests.get(self.url)

#         if response.status_code == 200:
#             return response
        
#         print(f"Error:{response.text}")

#     def show_meterdata(self) -> object:
#         pass

#     def get_total_power(self, meterdata: dict) -> float:
#         # get the list of 3 phase data
#         meterdata = meterdata['data']['values']

#         # get three phase powers from each phase
#         pa = meterdata[0][2]
#         pb = meterdata[1][2]
#         pc = meterdata[2][2]
#         total_power = pa+pb+pc 
#         return total_power
        

# # create meter instances for each smart meter
# physics_meter = Meter(URL[PHYSICS])
# biotech_meter = Meter(URL[BIOTECH])
# management_meter = Meter(URL[MANAGEMENT])
# civil_meter = Meter(URL[CIVIL])

# counter = 0

# # run an infinite loop for continuous data retrieval
# while True:
#     physics_meter_data = physics_meter.get_meterdata()
#     biotech_meter_data = biotech_meter.get_meterdata()
#     management_meter_data = management_meter.get_meterdata()
#     civil_meter_data = civil_meter.get_meterdata()

#     # # if it is not a cached result, sleep for 5 minutes - pause for 5 minutes before making another API call
#     # if not getattr(meter_data, 'from_cache', False):
#     #     time.sleep(WAIT_TIME)

#     # convert the data into json format
#     physics_meter_data = physics_meter_data.json()
#     biotech_meter_data = biotech_meter_data.json()
#     management_meter_data = management_meter_data.json()
#     civil_meter_data = civil_meter_data.json()
    
#     # get total three phase active power from each meter
#     physics_meter_total_power = physics_meter.get_total_power(physics_meter_data)
#     biotech_meter_total_power = biotech_meter.get_total_power(biotech_meter_data)
#     management_meter_total_power = management_meter.get_total_power(management_meter_data)
#     civil_meter_total_power = civil_meter.get_total_power(civil_meter_data)

#     network.loads.loc['Load5', 'p_set'] = management_meter_total_power/1e6
#     network.loads.loc['Load6', 'p_set'] = civil_meter_total_power/1e6
#     network.loads.loc['Load16', 'p_set'] = physics_meter_total_power/1e6
#     network.loads.loc['Load19', 'p_set'] = biotech_meter_total_power/1e6
    
#     # calculate reactive powers assuming PF = 0.95
#     network.loads.loc['Load5', 'q_set'] = (management_meter_total_power/1e6)*tan_phi
#     network.loads.loc['Load6', 'q_set'] = (civil_meter_total_power/1e6)*tan_phi
#     network.loads.loc['Load16', 'q_set'] = (physics_meter_total_power/1e6)*tan_phi
#     network.loads.loc['Load19', 'q_set'] = (biotech_meter_total_power/1e6)*tan_phi

#     # check for network consistency
#     network.consistency_check()
    
#     # perform newton Raphson Load Flow
#     network.pf()

    

#     # create a map object
#     map = folium.Map(location=(27.619013147338894, 85.5387356168638), 
#                      zoom_start=17, max_zoom=30)

#     # add a layer to plot the distribution grid
#     grid_layer = folium.FeatureGroup(name='Grid Layer').add_to(map)

#     # get coordinates of all the buses in the network
#     bus_coords = []
#     for index, row in network.buses.iterrows():
#     #     first latitude then longitude as folium expects location in this order
#         bus_coords.append([row['y'], row['x']])


#     # add circles to the locations of buses in the map
#     for i in range(len(bus_coords)):
#         # get the bus name
#         bus_name = network.buses.index.to_list()[i]
#         # get per unit voltage magnitude the bus
#         v_mag_pu = network.buses_t.v_mag_pu.iloc[0, i]
#         # get voltage angle of the bus (in radian by default) and convert it to degree
#         v_ang_rad = network.buses_t.v_ang.iloc[0, i]
#         v_ang_deg = (180/math.pi)*v_ang_rad 
#         # set bus color based on voltage magnitude
#         bus_color=''
#         if v_mag_pu<0.9:
#             bus_color='red'
#         elif 0.9<=v_mag_pu<0.95:
#             bus_color='violet'
#         elif 0.95<=v_mag_pu<=1.05:
#             bus_color='green'
#         elif 1.05<v_mag_pu<=1.1:
#             bus_color='orange'
#         else:
#             bus_color='yellow'
#         # show bus voltage magnitude and voltage angle on the popup 
#         popup_text = f'<span style="font-weight:bold; padding-left:20px;">{bus_name}</span><br>|V| = {v_mag_pu: .3f} p.u.<br>δ = {v_ang_deg: .3f} deg'
#         folium.Circle(location=bus_coords[i], radius=3.5, 
#                       stroke=False,
#                      fill=True, fill_color= bus_color, fill_opacity=1.0,
#                      popup=folium.Popup(popup_text, max_width=100)).add_to(grid_layer)


#     # add lines
#     for index, row in network.lines.iterrows():
#         # get the name of the line
#         line_name = index
#         # get active and reactive powers of the line
#         line_p = network.lines_t.p0.loc['now', index ]
#         line_q = network.lines_t.q0.loc['now', index ]    
#         # get the starting and ending buses of each line
#         bus0 = row['bus0']
#         bus1 = row['bus1']
#         # set line colors based on the line loading# assume nominal line apparent capacity as 0.4 MVA
#         s_nom_assumed = 0.069   #assumed nominal capacity of the line (230*300/1000000 MVA)
#         # calculate percentage loading of the line
#         percentage_loading = (abs(network.lines_t.p0.loc['now', index ])/s_nom_assumed)*100
#         line_color = ''
#         if percentage_loading <= 25:
#            # green color if line loading is less than 25%
#            line_color = 'green'
#         elif (25 < percentage_loading <= 50):
#             # blue if line loading is between 25 to 50%
#            line_color = 'blue' 
#         elif (50 < percentage_loading <= 75):
#             # violet if line loading is between 50 to 75%
#            line_color = 'orange' 
#         elif (75 < percentage_loading <= 100):
#             # orange if line loading is between 50 to 75%
#            line_color = 'violet'
#         else:
#             # red if line loading is greater than 100%
#            line_color = 'red'

#         # tooltip text for the line
#         tooltip_text = f'<span style="font-weight: bold; padding-left: 0px">{line_name}</span><br>P = {line_p: .3f} MW<br>Q = {line_q:.3f} MVAr'
#         # now, finally add the line
#         # latitude first then longitude
#         folium.PolyLine(locations=[(network.buses.loc[bus0].y, network.buses.loc[bus0].x), 
#                                   (network.buses.loc[bus1].y, network.buses.loc[bus1].x)],
#                         color = line_color,
#                        tooltip= tooltip_text).add_to(grid_layer)

#     # add line between HVB and LVB1 as PyPSA doesn't create a line between the buses if there is a transformer in between
#     folium.PolyLine(locations=[(network.buses.loc['HVB'].y, network.buses.loc['HVB'].x), 
#                                   (network.buses.loc['LVB1'].y, network.buses.loc['LVB1'].x)],
#                               color = 'black').add_to(grid_layer)

#     folium.LayerControl().add_to(map)

#     # Define a legend for the buses
#     bus_legend_html = """
#          <div style="position: fixed; 
#          top: 100px; right: 50px; width: 150px; height: 250px; 
#          border:0px solid grey; z-index:9999; font-size:14px;
#          background-color: white;
#          ">&nbsp; <span style="font-weight: bold; font-size: 20px">Bus Legends </span></b><br>
#          &nbsp; <font color="red" style="font-size: 30px;">●</font><span style="font-weight:bold;"> |V| < 0.9</span>   <br>
#          &nbsp; <font color="violet" style="font-size: 30px;">●</font><span style="font-weight:bold;"> 0.90 ≤ |V| < 0.95</span><br>
#          &nbsp; <font color="green" style="font-size: 30px;">●</font><span style="font-weight:bold;"> 0.95 ≤ |V| ≤ 1.05</span><br>
#          &nbsp; <font color="orange" style="font-size: 30px;">●</font><span style="font-weight:bold;"> 1.05 < |V| ≤ 1.10</span><br>
#          &nbsp; <font color="yellow" style="font-size: 30px;">●</font><span style="font-weight:bold;"> 1.10 < |V|</span><br>
#           </div>
#          """

#     # Define a legend for the lines
#     line_legend_html = """
#          <div style="position: fixed; 
#          bottom: 20px; right: 20px; width: 200px; height: 250px; 
#          border:0px solid grey; z-index:9999; font-size:14px;
#          background-color: white;
#          ">&nbsp; <span style="font-weight: bold; font-size: 20px">Line Legends </span></b><br>
#          &nbsp; <font color="green" style="font-size: 30px;">—</font><span style="font-weight:bold;"> Loading ≤ 25%</span><br>
#          &nbsp; <font color="blue" style="font-size: 30px;">—</font><span style="font-weight:bold;"> 25% ≤ Loading < 50%</span><br>
#          &nbsp; <font color="orange" style="font-size: 30px;">—</font><span style="font-weight:bold;"> 50% ≤ Loading < 75%</span><br>
#          &nbsp; <font color="violet" style="font-size: 30px;">—</font><span style="font-weight:bold;"> 75% ≤ Loading < 100%</span><br>
#          &nbsp; <font color="red" style="font-size: 30px;">—</font><span style="font-weight:bold;"> Loading > 100%</span><br>
#           </div>
#          """

#     # Add bus legend to the map
#     map.get_root().html.add_child(folium.Element(bus_legend_html))

#     # Add line legend to the map
#     map.get_root().html.add_child(folium.Element(line_legend_html))
    
#     map.save('ku_grid.html')
    
# #     Autorefresh section -- modify the html file so that it autorefreshes every 5 minutes
#     with open('ku_grid.html', 'r', encoding='utf-8') as f:
#         f_contents = f.read()
    
#     refreshed_content = f_contents.replace('</head>', '<meta http-equiv="refresh" content="300"></head>')

#     with open('ku_grid.html', 'w', encoding='utf-8') as f:
#         f.write(refreshed_content)

#     # clear the previous figure beofre plotting the new figure
# #     clear_output(wait=True)

#     # wait before making another api call
#     time.sleep(WAIT_TIME)


<h3>Plotting with pypsa.Network.Plot() - No Longer Used</h3>

In [ ]:
# import requests
# import json
# import time
# from IPython.display import display, clear_output

# TOTAL_BLOCKS = 4

# PHYSICS = 0
# BIOTECH = 1
# MANAGEMENT = 2
# CIVIL = 3

# # set wait time(in seconds) before making another API call
# WAIT_TIME = 5*60

# # SN of energy meters installed in each block
# # create a list of random strings for initialization
# SN = ["initial_SN_string" for _ in range(TOTAL_BLOCKS)]

# # assign SN of individual meters
# SN[PHYSICS] = 'CD0FF6AB'
# SN[BIOTECH] = '57DB095D'
# SN[MANAGEMENT] = '8FA834AC'
# SN[CIVIL] = 'DAD94549'

# # TOKEN to be used for making api call
# TOKEN = 'a971224e6dc04edfa62bc0e83978d854'

# # create URLs to make API calls to fetch data from the Iammeter Cloud
# URL = []
# for i in range(TOTAL_BLOCKS):
#     URL.append("http://www.iammeter.com/api/v1/site/meterdata2/" + SN[i] + "?token=" + TOKEN)


# class Meter:
#     def __init__(self, url: str):
#         self.url = url
#         # print(self.url)

#     def get_meterdata(self) -> object:
#         response = requests.get(self.url)

#         if response.status_code == 200:
#             return response
        
#         print(f"Error:{response.text}")

#     def show_meterdata(self) -> object:
#         pass

#     def get_total_power(self, meterdata: dict) -> float:
#         # get the list of 3 phase data
#         meterdata = meterdata['data']['values']

#         # get three phase powers from each phase
#         pa = meterdata[0][2]
#         pb = meterdata[1][2]
#         pc = meterdata[2][2]
#         total_power = pa+pb+pc 
#         return total_power
        

# # create meter instances for each smart meter
# physics_meter = Meter(URL[PHYSICS])
# biotech_meter = Meter(URL[BIOTECH])
# management_meter = Meter(URL[MANAGEMENT])
# civil_meter = Meter(URL[CIVIL])

# # run an infinite loop for continuous data retrieval
# while(1):
#     physics_meter_data = physics_meter.get_meterdata()
#     biotech_meter_data = biotech_meter.get_meterdata()
#     management_meter_data = management_meter.get_meterdata()
#     civil_meter_data = civil_meter.get_meterdata()

#     # # if it is not a cached result, sleep for 5 minutes - pause for 5 minutes before making another API call
#     # if not getattr(meter_data, 'from_cache', False):
#     #     time.sleep(WAIT_TIME)

#     # convert the data into json format
#     physics_meter_data = physics_meter_data.json()
#     biotech_meter_data = biotech_meter_data.json()
#     management_meter_data = management_meter_data.json()
#     civil_meter_data = civil_meter_data.json()
    
#     # get total three phase active power from each meter
#     physics_meter_total_power = physics_meter.get_total_power(physics_meter_data)
#     biotech_meter_total_power = biotech_meter.get_total_power(biotech_meter_data)
#     management_meter_total_power = management_meter.get_total_power(management_meter_data)
#     civil_meter_total_power = civil_meter.get_total_power(civil_meter_data)

#     network.loads.loc['Load5', 'p_set'] = management_meter_total_power/1e6
#     network.loads.loc['Load6', 'p_set'] = civil_meter_total_power/1e6
#     network.loads.loc['Load16', 'p_set'] = physics_meter_total_power/1e6
#     network.loads.loc['Load19', 'p_set'] = biotech_meter_total_power/1e6
    
#     # calculate reactive powers assuming PF = 0.95
#     network.loads.loc['Load5', 'q_set'] = (management_meter_total_power/1e6)*tan_phi
#     network.loads.loc['Load6', 'q_set'] = (civil_meter_total_power/1e6)*tan_phi
#     network.loads.loc['Load16', 'q_set'] = (physics_meter_total_power/1e6)*tan_phi
#     network.loads.loc['Load19', 'q_set'] = (biotech_meter_total_power/1e6)*tan_phi

#     # check for network consistency
#     network.consistency_check()
    
#     # perform newton Raphson Load Flow
#     network.pf()


#     # adjust bus colors
#     # the first element of bus_colors correspond to HVB. since it is set as a slack bus, 
#     # its color is set to cadetblue
#     bus_colors = ["#5F9EA0"]
#     bus_names = network.buses.index.to_list()
#     for i in range(1, 53):
#        bus_voltage_pu = network.buses_t.v_mag_pu.loc['now',f'LVB{i}']
#        if 0.95 < bus_voltage_pu <= 1.05:
#            # cadetblue color if bus voltage is between 0.95 and 1.05pu
#            bus_colors.append("#5F9EA0")
#        elif (0.9 < bus_voltage_pu <= 0.95) or ((1.05 < bus_voltage_pu <= 1.1)):
#            # sky blue if bus voltage is between 0.9 and 0.95 pu
#            bus_colors.append("#1DA1E8") 
#        else:
#            # red if bus voltage is less than 0.9pu or greater than 1.1 pu
#            bus_colors.append("#F15151")   
#     bus_colors = pd.Series(bus_colors, index=bus_names) 

    

#     # adjust line colors
#     # assume nominal line apparent capacity as 0.4 MVA
#     s_nom_assumed = 0.4
#     # calculate percentage loading
#     percentage_loading = (abs(network.lines_t.p0.loc['now'])/s_nom_assumed)*100
#     percentage_loading = abs(percentage_loading)

    
#     fig, ax = plt.subplots(subplot_kw={"projection":ccrs.PlateCarree()}, figsize = (10,10))
#     collection = network.plot(ax = ax, 
#                 bus_sizes = 0.0000000012, bus_colors = bus_colors,
#                 line_widths=1.50, line_colors=percentage_loading, line_cmap=plt.cm.turbo, line_alpha = 1.0,
#                  projection = ccrs.EqualEarth(),
#                 flow = None,
#                 )
#     heatmap = plt.colorbar(collection[2], 
#                  fraction = 0.035, pad = 0.008, 
#                  ticks = [0, 25, 50, 75, 100, 125, 150, 175],
#                  label = "Line Percentage loading",
#                 )
#     heatmap.ax.set_yticklabels(['0', '25', '50', '75', '100', '125', '150', '175'])
    
#     # print bus names alongside each bus
#     for bus in network.buses.index:
#         # print bus names slightly above the bus to avoid overlap
#         ax.text(network.buses.loc[bus]['x']-0.00006, network.buses.loc[bus]['y']+0.00006, f"{bus}", fontsize = 7)
    
#     # save figure in png format
#     plt.savefig('KU_flow_plot.png')
    
#     display(plt.gcf())

#     # clear the previous figure beofre plotting the new figure
#     clear_output(wait=True)

#     # wait before making another api call
#     time.sleep(WAIT_TIME)
